# k - Nearest Neighbors

In [20]:
#install.packages("caTools")
library("ISLR")
library("caTools")
library('rsample')  # data splitting
library("klaR") # naive bayes
library("caret") 
library("dplyr")
library("class") # knn
library("ggplot2")
options(repr.plot.width = 16, repr.plot.height = 6)

In [3]:
str(Caravan)

'data.frame':	5822 obs. of  86 variables:
 $ MOSTYPE : num  33 37 37 9 40 23 39 33 33 11 ...
 $ MAANTHUI: num  1 1 1 1 1 1 2 1 1 2 ...
 $ MGEMOMV : num  3 2 2 3 4 2 3 2 2 3 ...
 $ MGEMLEEF: num  2 2 2 3 2 1 2 3 4 3 ...
 $ MOSHOOFD: num  8 8 8 3 10 5 9 8 8 3 ...
 $ MGODRK  : num  0 1 0 2 1 0 2 0 0 3 ...
 $ MGODPR  : num  5 4 4 3 4 5 2 7 1 5 ...
 $ MGODOV  : num  1 1 2 2 1 0 0 0 3 0 ...
 $ MGODGE  : num  3 4 4 4 4 5 5 2 6 2 ...
 $ MRELGE  : num  7 6 3 5 7 0 7 7 6 7 ...
 $ MRELSA  : num  0 2 2 2 1 6 2 2 0 0 ...
 $ MRELOV  : num  2 2 4 2 2 3 0 0 3 2 ...
 $ MFALLEEN: num  1 0 4 2 2 3 0 0 3 2 ...
 $ MFGEKIND: num  2 4 4 3 4 5 3 5 3 2 ...
 $ MFWEKIND: num  6 5 2 4 4 2 6 4 3 6 ...
 $ MOPLHOOG: num  1 0 0 3 5 0 0 0 0 0 ...
 $ MOPLMIDD: num  2 5 5 4 4 5 4 3 1 4 ...
 $ MOPLLAAG: num  7 4 4 2 0 4 5 6 8 5 ...
 $ MBERHOOG: num  1 0 0 4 0 2 0 2 1 2 ...
 $ MBERZELF: num  0 0 0 0 5 0 0 0 1 0 ...
 $ MBERBOER: num  1 0 0 0 4 0 0 0 0 0 ...
 $ MBERMIDD: num  2 5 7 3 0 4 4 2 1 3 ...
 $ MBERARBG: num  5 0 0 

In [4]:
summary(Caravan$Purchase)

No  Yes 
5474  348

In [5]:
# check NA values
any(is.na(Caravan))

[1] FALSE

In [9]:
set.seed(123)
# stratified split 70% for training, and the rest for testing
split <- initial_split(Caravan, prop = 0.7, strata = "Purchase")
train <- training(split)
test  <- testing(split)

In [10]:
# distribution of train
table(train$Purchase) 


  No  Yes 
3825  250 

In [11]:
# distribution of test set
table(test$Purchase)


  No  Yes 
1649   98 

In [17]:
# create stratified training and testing
features <- setdiff(names(train), "Purchase")
# training
x_train <- train[, features]
y_train <- train$Purchase
# testing
x_test <- test[,features]
y_test <- test$Purchase

In [21]:
# set up 10-fold cross validation procedure
train_control <- trainControl(method = "cv", number = 10)

In [22]:
# train model
knn <- train(x = x_train, 
            y = y_train, 
            method = "knn", 
            trControl = train_control,
            preProc = c("scale"))

Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: PZEILPL, AZEILPL"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: PZEILPL, AZEILPL"
Warning message in preProcess.default(thresh = 0.95, k = 5, freqCut = 19, uniqueCut = 10, :
"These variables have zero variances: PZEILPL, AZEILPL"


In [23]:
knn

k-Nearest Neighbors 

4075 samples
  85 predictor
   2 classes: 'No', 'Yes' 

Pre-processing: scaled (85) 
Resampling: Cross-Validated (10 fold) 
Summary of sample sizes: 3668, 3668, 3667, 3667, 3668, 3668, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa     
  5  0.9330045  0.03684649
  7  0.9366876  0.01604664
  9  0.9386508  0.01356210

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 9.

## Prediction

In [24]:
y_pred <- predict(knn, x_test, type="raw")
test_y_pred <- test %>% mutate('y_pred'=y_pred) 
head(test_y_pred)


,MOSTYPE,MAANTHUI,MGEMOMV,MGEMLEEF,MOSHOOFD,MGODRK,MGODPR,MGODOV,MGODGE,MRELGE,...,AGEZONG,AWAOREG,ABRAND,AZEILPL,APLEZIER,AFIETS,AINBOED,ABYSTAND,Purchase,y_pred
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,...,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
3,37,1,2,2,8,0,4,2,4,3,...,0,0,1,0,0,0,0,0,No,No
4,9,1,3,3,3,2,3,2,4,5,...,0,0,1,0,0,0,0,0,No,No
6,23,1,2,1,5,0,5,0,5,0,...,0,0,0,0,0,0,0,0,No,No
8,33,1,2,3,8,0,7,0,2,7,...,0,0,0,0,0,0,0,0,No,No
9,33,1,2,4,8,0,1,3,6,6,...,0,0,0,0,0,0,0,0,No,No
11,10,1,4,3,3,1,4,1,4,7,...,0,0,0,0,0,0,0,0,No,No


## Evaluation

In [25]:
confusionMatrix(y_pred, y_test)

Confusion Matrix and Statistics

          Reference
Prediction   No  Yes
       No  1647   96
       Yes    2    2
                                          
               Accuracy : 0.9439          
                 95% CI : (0.9321, 0.9542)
    No Information Rate : 0.9439          
    P-Value [Acc > NIR] : 0.5268          
                                          
                  Kappa : 0.035           
                                          
 Mcnemar's Test P-Value : <2e-16          
                                          
            Sensitivity : 0.99879         
            Specificity : 0.02041         
         Pos Pred Value : 0.94492         
         Neg Pred Value : 0.50000         
             Prevalence : 0.94390         
         Detection Rate : 0.94276         
   Detection Prevalence : 0.99771         
      Balanced Accuracy : 0.50960         
                                          
       'Positive' Class : No              
                        